# Reproducing M2E2 using a prompt-based LLM
DFKI | Rayyan M

---

### TO-DO
- Individual JSON files for text-only, image-only, and separate for multimedia
- For image-only, make sure the text for the image is captured through the captions.json
- Fix the example formats (look at annotations.zip)


### Defining schema/ontology

In [ ]:
# event obj : argument roles from M2E2 dataset paper

event_arg = {
    "life.die":["agent", "victim", "instrument", "place"],
    "movement.transport":["destination", "origin", "instrument", "agent", "artifact/person"],
    "transaction.transfermoney":["giver","recipient", "money"],
    "conflict.attack":["attacker", "instrument", "place", "target"],
    "conflict.demonstrate":["demonstrator", "instrument", "police", "place"],
    "contact.meet":["participant", "place"],
    "contact.phone-write":["participant", "instrument", "place"],
    "justice.arrestjail":["agent", "person", "instrument", "place"]
}

In [3]:
from pathlib import Path
import shutil

# 1) Set the article filename here (must exist under m2e2_rawdata/article)
article_filename = "VOA_EN_NW_2009.12.09.416313.rsd.txt"  # change this

# 2) Option: change matching behavior if needed
strict_prefix_match = True  # if False, will also match images that merely contain the base name

# --- Paths ---
project_root = Path.cwd()
article_dir = project_root / "m2e2_rawdata" / "article"
images_dir = project_root / "m2e2_rawdata" / "image" / "image"

# Validate paths
article_path = article_dir / article_filename
if not article_path.exists():
    raise FileNotFoundError(f"Article file not found: {article_path}")
if not images_dir.exists():
    raise FileNotFoundError(f"Images directory not found: {images_dir}")

# Read the article text into a variable
article_text = article_path.read_text(encoding="utf-8", errors="ignore")

# Derive a base name by stripping .txt and optional .rsd suffix
base_name = article_filename
if base_name.endswith(".txt"):
    base_name = base_name[:-4]
if base_name.endswith(".rsd"):
    base_name = base_name[:-4]


output_root = project_root / "output"
base_output_dir = output_root / base_name
articles_output_dir = base_output_dir / "articles"
images_output_dir = base_output_dir / "images"
articles_output_dir.mkdir(parents=True, exist_ok=True)
images_output_dir.mkdir(parents=True, exist_ok=True)

# Copy the article file into articles subfolder
shutil.copy2(article_path, articles_output_dir / article_filename)

# Find corresponding images
matched_images = []
for img_path in images_dir.glob("*.jpg"):
    name = img_path.name
    if strict_prefix_match:
        if name.startswith(base_name):
            matched_images.append(img_path)
    else:
        if base_name in name:
            matched_images.append(img_path)

# Copy images into the images subfolder
for img in matched_images:
    shutil.copy2(img, images_output_dir / img.name)

# Create an empty JSON file at the base folder level
json_path = base_output_dir / f"OUTPUT_{base_name}.json"
json_path.write_text("", encoding="utf-8")

print(f"Article text length: {len(article_text)} characters")
print(f"Base folder: {base_output_dir}")
print(f"Article copied to: {articles_output_dir / article_filename}")
print(f"Images copied: {len(matched_images)} into {images_output_dir}")
print(f"Empty JSON created at: {json_path}")


Article text length: 1205 characters
Base folder: c:\Users\rayya\Desktop\DFKI\M2E2\output\VOA_EN_NW_2009.12.09.416313
Article copied to: c:\Users\rayya\Desktop\DFKI\M2E2\output\VOA_EN_NW_2009.12.09.416313\articles\VOA_EN_NW_2009.12.09.416313.rsd.txt
Images copied: 3 into c:\Users\rayya\Desktop\DFKI\M2E2\output\VOA_EN_NW_2009.12.09.416313\images
Empty JSON created at: c:\Users\rayya\Desktop\DFKI\M2E2\output\VOA_EN_NW_2009.12.09.416313\OUTPUT_VOA_EN_NW_2009.12.09.416313.json


## (i) Text-only extraction

In WASE, each sentence is paired with the most relevant image (via embedding similarity). Right now we are choosing to not do that but in future we will: Either ask the LLM to select which image best matches a sentence (if you can pass both text + images together), or compute a similarity score (text embedding vs image embedding) and pick the closest image.

TO-DO: Pass images in input and ask prompt to select images that best match

In [4]:
example_json = [
    {
        "sentence_id": 1,
        "text": "UNECA Director Says Dangers in Guinea are Serious",
        "events": [
            {
                "event_type": "Conflict.Attack",
                "modality": "text",
                "trigger": {"text": "Dangers", "char_start": 22, "char_end": 28},
                "arguments": [
                    {"role": "Place", "text": "Guinea", "char_start": 32, "char_end": 37}
                ]
            }
        ]
    },
    {
        "sentence_id": 2,
        "text": "Bodies of people killed during a rally are seen at the capital's main mosque in Conakry, Guinea",
        "events": [
            {
                "event_type": "Conflict.Attack",
                "modality": "text",
                "trigger": {"text": "killed", "char_start": 20, "char_end": 26},
                "arguments": [
                    {"role": "Victim", "text": "people", "char_start": 10, "char_end": 16},
                    {"role": "Place", "text": "Conakry, Guinea", "char_start": 82, "char_end": 96}
                ]
            }
        ]
    }
]


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json

model_name = "google/gemma-3-1b-it"  # smaller version (270m) - performance was not good at all
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.bfloat16, 
    device_map="auto")


prompt = f"""
you are an information extraction system. Extract events and arguments for each sentence from the following article: {article_text}.

RULES:
1. Only use the following event types and their argument roles: {event_arg}
2. Output **must be a valid JSON array** only, nothing else. **Do not add markdown, backslashes, escape characters, or extra text**.
3. Only use double quotation marks (") for JSON strings.
4. Each event must include:
    a. "sentence id": an iterator over all the sentences
    b. "text": the sentence itself
    c. "events": these further contain the following:
        a. "event_type": the event type string.
        b. "modality": always text.
        c. "trigger": the word(S) that signal the event, with "text", "char_start", "char_end".
        d. "arguments": a list of objects, each with:
            i. "role": role from the ontology.
            ii. "text" the argument string.
            iii. "char_start": start character index of the argument string.
            iv. "char_end": end character index of the argument string.
5. Offsets are character indices in the sentence (0-based, inclusive-exclusive).
6. **Do not include explanations, notes, comments, or any text outside the JSON array.**
7. The output must be fully parseable by `json.loads()` in Python.


Here is an example output, your output should follow this exact JSON format, there can be more or less sentence id depending on the article: {example_json}

"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

output_ids = model.generate(
    **inputs,
    max_new_tokens=1024,  # adjust based on expected length
    do_sample=False  # deterministic output
)

#generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
generated_ids = output_ids[0][inputs['input_ids'].shape[1]:]
generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [6]:
generated_text

'```json\n[\n  {\'sentence_id\': 1, \'text\': \'UNECA Director Says Dangers in Guinea are Serious\', \'events\': [{\'event_type\': \'Conflict.Attack\', \'modality\': \'text\', \'trigger\': {\'text\': \'Dangers\', \'char_start\': 22, \'char_end\': 28}, \'arguments\': [{\'role\': \'Place\', \'text\': \'Guinea\', \'char_start\': 32, \'char_end\': 37}]}]},\n  {\'sentence_id\': 2, \'text\': "Bodies of people killed during a rally are seen at the capital\'s main mosque in Conakry, Guinea", \'events\': [{\'event_type\': \'Conflict.Attack\', \'modality\': \'text\', \'trigger\': {\'text\': \'killed\', \'char_start\': 20, \'char_end\': 26}, \'arguments\': [{\'role\': \'Victim\', \'text\': \'people\', \'char_start\': 10, \'char_end\': 16}, {\'role\': \'Place\', \'text\': \'Conakry, Guinea\', \'char_start\': 82, \'char_end\': 96}]}]}\n]\n```\n'

In [7]:
import ast
import json

cleaned_text = generated_text.strip('```json\n').strip('```\n')

python_obj = ast.literal_eval(cleaned_text)

json_data = json.dumps(python_obj, indent=2)

print(json_data)

[
  {
    "sentence_id": 1,
    "text": "UNECA Director Says Dangers in Guinea are Serious",
    "events": [
      {
        "event_type": "Conflict.Attack",
        "modality": "text",
        "trigger": {
          "text": "Dangers",
          "char_start": 22,
          "char_end": 28
        },
        "arguments": [
          {
            "role": "Place",
            "text": "Guinea",
            "char_start": 32,
            "char_end": 37
          }
        ]
      }
    ]
  },
  {
    "sentence_id": 2,
    "text": "Bodies of people killed during a rally are seen at the capital's main mosque in Conakry, Guinea",
    "events": [
      {
        "event_type": "Conflict.Attack",
        "modality": "text",
        "trigger": {
          "text": "killed",
          "char_start": 20,
          "char_end": 26
        },
        "arguments": [
          {
            "role": "Victim",
            "text": "people",
            "char_start": 10,
            "char_end": 16
          },


In [8]:
from pathlib import Path

try:
    out_path = json_path  # from earlier cell
except NameError:
    project_root = Path.cwd()
    base_output_dir = project_root / "output" / base_name
    base_output_dir.mkdir(parents=True, exist_ok=True)
    out_path = base_output_dir / f"OUTPUT_{base_name}.json"

with open(out_path, "w", encoding="utf-8") as f:
    f.write(json_data)

print(f"Saved JSON to: {out_path}")

Saved JSON to: c:\Users\rayya\Desktop\DFKI\M2E2\output\VOA_EN_NW_2009.12.09.416313\OUTPUT_VOA_EN_NW_2009.12.09.416313.json


## (ii) Image-only extraction

In [4]:
example_json_img = [
  {
    "image_id": "Actual_Image_Name.jpg",
    "events": [
      {
        "event_type": "Conflict.Attack",
        "modality": "image",
        "trigger": {"text": "attack"}, 
        "arguments": [
          {
            "role": "Attacker",
            "text": "soldiers",
            "bbox": [0.12, 0.40, 0.45, 0.78]
          },
          {
            "role": "Victim",
            "text": "protesters",
            "bbox": [0.50, 0.35, 0.80, 0.70]
          },
          {
            "role": "Place",
            "text": "Conakry",
            "bbox": [0.05, 0.10, 0.30, 0.25]
          }
        ]
      }
    ]
  }
]

In [ ]:
from transformers import AutoModelForCausalLM, AutoProcessor
import torch
from PIL import Image


model_name = "google/gemma-3-4b-it"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="cpu", offload_folder="offload")



# Example: load one image
img_path = images_output_dir / matched_images[1].name  # from earlier matching
#img_path = "output\VOA_EN_NW_2009.12.09.416313\images\VOA_EN_NW_2009.12.09.416313_0.jpg"
image = Image.open(img_path).convert("RGB")

prompt = f"""
<start_of_image><end_of_image>

You are an event extraction system. Analyze the image called {matched_images[1].name} and extract events.

RULES:
1. Only use the following event types and their roles: {event_arg}
2. Output **must be a valid JSON array** only, nothing else. **Do not add markdown, backslashes, escape characters, or extra text**.
3. Each event must include:
   - "event_type": the event type.
   - "modality": always "image".
   - "trigger": a short word/phrase describing the main event (string only).
   - "arguments": a list of objects, each with:
       * "role": role name
       * "text": short phrase for the entity (e.g., "soldier", "protesters", "gun", "car").
       * "bbox": [x_min, y_min, x_max, y_max] with normalized coordinates between 0 and 1.
4. If no clear event is present, return an empty list [].
5. Do not add notes, markdown, or comments — only strict JSON.


here is an example 

Here is an example output, your output should follow this exact JSON format: {example_json_img}



"""

inputs = processor(text=prompt, images=[image], return_tensors="pt").to(model.device)

# Generate
generated_ids = model.generate(
    **inputs,
    max_new_tokens=512,
    do_sample=False
)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [21]:
print(processor.tokenizer.special_tokens_map)
print(processor.tokenizer.additional_special_tokens)

{'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'boi_token': '<start_of_image>', 'eoi_token': '<end_of_image>', 'image_token': '<image_soft_token>'}
[]


In [12]:
input_length = inputs["input_ids"].shape[1]
generated_text = processor.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0]

In [13]:
generated_text

'```json\n[\n  {\n    "image_id": "VOA_EN_NW_2009.12.09.416313_1.jpg",\n    "events": [\n      {\n        "event_type": "life.die",\n        "modality": "image",\n        "trigger": "death",\n        "arguments": [\n          {\n            "role": "victim",\n            "text": "corpses",\n            "bbox": [\n              0.0,\n              0.1,\n              0.6,\n              0.95\n            ]\n          },\n          {\n            "role": "place",\n            "text": "ground",\n            "bbox": [\n              0.0,\n              0.0,\n              1.0,\n              0.3\n            ]\n          }\n        ]\n      },\n      {\n        "event_type": "conflict.demonstrate",\n        "modality": "image",\n        "trigger": "police",\n        "arguments": [\n          {\n            "role": "police",\n            "text": "police",\n            "bbox": [\n              0.3,\n              0.2,\n              0.8,\n              0.9\n            ]\n          }\n      

In [14]:
import ast
import json

cleaned_text = generated_text.strip('```json\n').strip('```\n')

python_obj = ast.literal_eval(cleaned_text)

json_data = json.dumps(python_obj, indent=2)

print(json_data)

[
  {
    "image_id": "VOA_EN_NW_2009.12.09.416313_1.jpg",
    "events": [
      {
        "event_type": "life.die",
        "modality": "image",
        "trigger": "death",
        "arguments": [
          {
            "role": "victim",
            "text": "corpses",
            "bbox": [
              0.0,
              0.1,
              0.6,
              0.95
            ]
          },
          {
            "role": "place",
            "text": "ground",
            "bbox": [
              0.0,
              0.0,
              1.0,
              0.3
            ]
          }
        ]
      },
      {
        "event_type": "conflict.demonstrate",
        "modality": "image",
        "trigger": "police",
        "arguments": [
          {
            "role": "police",
            "text": "police",
            "bbox": [
              0.3,
              0.2,
              0.8,
              0.9
            ]
          }
        ]
      }
    ]
  }
]


### The text here needs to be taken from the captions file, instead of asking LLM to generate!

In [15]:
import json
from pathlib import Path

try:
    out_path = json_path  # from earlier cell
except NameError:
    project_root = Path.cwd()
    base_output_dir = project_root / "output" / base_name
    base_output_dir.mkdir(parents=True, exist_ok=True)
    out_path = base_output_dir / f"OUTPUT_{base_name}.json"

if out_path.exists():
    with open(out_path, "r", encoding="utf-8") as f:
        existing_data = json.load(f)
else:
    existing_data = []

new_data = json.loads(json_data)  # convert string to Python object
if isinstance(new_data, list):
    existing_data.extend(new_data)
else:
    existing_data.append(new_data)

# Save back to file
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(existing_data, f, ensure_ascii=False, indent=2)

print(f"Appended JSON to: {out_path}")


Appended JSON to: c:\Users\rayya\Desktop\DFKI\M2E2\output\VOA_EN_NW_2009.12.09.416313\OUTPUT_VOA_EN_NW_2009.12.09.416313.json


## (iii) Cross-media alignment Step

We need to pair the right image to the right sentence, which means not relying on event type alone (Conflict.Attack is too broad for ex). An article can mention multiple attacks, and we need to pair the correct image. For this we will add a second filter for similarity score i.e, both must be the same event type AND semantically close. We use a transformer for semantic matching between text and image descriptions. 

In [19]:
#Going to use all-MiniLM-L6-v2 because its a small/fast snetence embedding model that maps text into dense vector. Then we can compute cosine similarity b/w 2 embeddings.
#Ex - "killed people Guinea" (from text) and "attack protesters Conakry" (from image) -----> High similarity because both talk about violence in Guinea.

from sentence_transformers import SentenceTransformer, util

embedder = SentenceTransformer("all-MiniLM-L6-v2")



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\rayya\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rayya\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [35]:
def comp_similarity(text_event, image_event):

    if text_event["event_type"].lower() != image_event["event_type"].lower():
        return 0.0
    
    else:
        text_summary = text_event["trigger"]["text"] + " " + " ".join(a["text"] for a in text_event["arguments"])
        image_summary = image_event["trigger"]["text"] + " " + " ".join([a["text"] for a in image_event["arguments"]])

        emb1 = embedder.encode(text_summary, convert_to_tensor=True)
        emb2 = embedder.encode(image_summary, convert_to_tensor=True)
        sim = util.pytorch_cos_sim(emb1, emb2).item()

        return sim


def align_events(text_events, image_events, threshold=0.0):
    multimedia_events=[]

    for txt_event in text_events:
        for img_event in image_events:
            score = comp_similarity(txt_event, img_event)

            if score > threshold:
                multimedia_events.append({
                    "event_type": txt_event["event_type"],
                    "modality": "multimedia",
                    "similarity": score,
                    "text_event": txt_event,
                    "image_event": img_event
                })

    
    return multimedia_events


In [36]:
project_root = Path.cwd()
base_output_dir = project_root / "output" / "VOA_EN_NW_2009.12.09.416313"
json_path = base_output_dir / "OUTPUT_VOA_EN_NW_2009.12.09.416313.json"


with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)


text_events = []
image_events = []

for entry in data:
    if "sentence_id" in entry:  # text block
        for ev in entry.get("events", []):
            if ev.get("modality") == "text":
                text_events.append(ev)
    
    elif "image_id" in entry:  # image block
        for ev in entry.get("events", []):
            if ev.get("modality") == "image":
                image_events.append(ev)


multimedia_events = align_events(text_events, image_events)


data.extend(multimedia_events)


with open(json_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"Appended {len(multimedia_events)} multimedia events to {json_path}")


Appended 2 multimedia events to c:\Users\rayya\Desktop\DFKI\M2E2\output\VOA_EN_NW_2009.12.09.416313\OUTPUT_VOA_EN_NW_2009.12.09.416313.json
